In [27]:
import nltk
import time
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import r2_score


In [28]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    featureList = []
    fea_vect=[]
    
    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            #self.data_model = pd.read_csv(from_cached)
            self.data_model = pd.read_json(from_cached)

            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            #self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = pd.read_json(csv_file)

            #self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
        
    
    
    
    def do_process(self):
        start_time = time.time()
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)
             
                def to_lower(text):
                    return text.lower()

                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    text=re.sub(r'[?|*|.|!|+|-]',r'',text)
                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                        

                    whitelist=["to","on","n't","not","don't","for","up","below","short","long"]

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if ((w not in stop_words) or (w in whitelist)) :
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
                #convert to string
                idx=row["spans"]
                #ch="".join(x for x in idx if x)
                ch=' '.join(idx)
               
    
                
                
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)

                txt_low=to_lower(txt_num)
                
                txt_ws=remove_whitespace(txt_low)
    

                txt_pun=remove_punctuations(txt_ws)

                txt_final=remove_stopwords(txt_pun)

                #texte=words_lemmatizer(txt_final)

                n_grams=get_ngrams(txt_final, 1)

                row["token_spans"] = n_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))

    

    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       ):
        def get_tweets(data):
            start_time = time.time()

            tweets=list(data.processed_data.token_spans)
    
            list_tweet=[]
            for i,chaine in enumerate(tweets):
                ch=" ".join(chaine)
                list_tweet.append(ch)
            print("--- %s seconds ---" % (time.time() - start_time))
    
            return list_tweet
    
        def get_top_n_words(corpus, n=None):
   
            vec = CountVectorizer().fit(corpus)
            bag_of_words = vec.transform(corpus)
            sum_words = bag_of_words.sum(axis=0) 
            words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
            words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
            return words_freq[:n]
        whitelist=[]
        stopwords=[]
        self.wordlist = []
        """
        corpus=get_tweets(self)
        common_words = get_top_n_words(corpus)
        word_df = pd.DataFrame(data={"word": [k for k, v in common_words],
                                     "occurrences": [v for k, v in common_words]},
                               columns=["word", "occurrences"])
        
        
        word_df.to_csv("wordlist_count.csv", index_label="idx")
        self.wordlist = [k for k, v in common_words]
        
        
        """
        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        

        
                
        print(words.most_common())
        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_copie.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        label_column = []
        Id_column=["ID"]
        label_column = ["label"]

        columns = Id_column + label_column + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if True:
                # add label
                current_label = self.processed_data.loc[idx, "sentiment score"]
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                labels.append(current_label)
                
                current_row.append(current_id)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels


In [29]:
class TwitterData_Initialize_test(TwitterData_Initialize):

    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"].split()))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)

                def to_lower(text):
                    return text.lower()
                
                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    text=re.sub(r'[?|*|.|!|+|-]',r'',text)

                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):

                    whitelist=["to","on","n't","not","don't","for","up","below","short","long"]

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if ((w not in stop_words) or (w in whitelist)) :
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
            
                idx=row["spans"]
                ch=' '.join(idx)


    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)
                
                
                txt_low=to_lower(txt_num)
                
                txt_ws=remove_whitespace(txt_low)
                    
    
                txt_pun=remove_punctuations(txt_ws)
        
                txt_final=remove_stopwords(txt_pun)

                #texte=words_lemmatizer(txt_final)

                n_grams=get_ngrams(txt_final, 1)
                #print("list ngrams--------------\n",n_grams)

                row["token_spans"] = n_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                      ):
        def get_tweets(data):
            start_time = time.time()

            tweets=list(data.processed_data.token_spans)
    
            list_tweet=[]
            for i,chaine in enumerate(tweets):
                ch=" ".join(chaine)
                list_tweet.append(ch)
            print("--- %s seconds ---" % (time.time() - start_time))
    
            return list_tweet
    
        def get_top_n_words(corpus, n=None):
   
            vec = CountVectorizer().fit(corpus)
            bag_of_words = vec.transform(corpus)
            sum_words = bag_of_words.sum(axis=0) 
            words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
            words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

            return words_freq[:n]

        whitelist=[]
        stopwords=[]
        self.wordlist = []
        """
        corpus=get_tweets(self)
        common_words = get_top_n_words(corpus)
        word_df = pd.DataFrame(data={"word": [k for k, v in common_words],
                                     "occurrences": [v for k, v in common_words]},
                               columns=["word", "occurrences"])
        
        
        word_df.to_csv("wordlist_count_test.csv", index_label="idx")
        self.wordlist = [k for k, v in common_words]
        
        
        
        """

        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        

        
                
        print(words.most_common())

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_test.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        
        label_id = ["ID"]

        columns = label_id + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []
            if True:
                # add label
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                
                current_row.append(current_id)

            

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        return self.data_model


In [30]:
data = TwitterData_Initialize()
data.initialize("Microblog_Trainingdata.json")

data.processed_data.head(20)

,cashtag,id,sentiment score,source,spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow]
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015]
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers]
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.]


In [31]:
data.do_process()

--- 2.8729028701782227 seconds ---


In [32]:
data.processed_data.head()

,cashtag,id,sentiment score,source,spans,token_spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watching, for, bounce, tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record, number, passengers, served]"
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35],[nflx]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ...","[looking, for, strong, bounce, lunchtime, rall..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...,"[intrigued, technology, growth, potential]"


In [33]:
data_test = TwitterData_Initialize_test()
data_test.initialize("Microblog_Trialdata.json")

data_test.processed_data.head()
#test_data=data_test.processed_data.copy()
data_test.do_process()
data_test.processed_data.head()


,cashtag,id,sentiment score,source,spans,token_spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short],"[putting, on, little, f, short]"
1,$AAPL,10752226,-0.464,stocktwits,[short some],[short]
2,$BAC,10920221,0.445,stocktwits,[buying opportunity],"[buying, opportunity]"
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position],"[scaling, up, on, long, position]"
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks],"[time, to, sell, banks]"


In [34]:
# Importing the built-in logging module
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [35]:
#list_tweet=get_tweets(data)
list_tweet=list(data.processed_data["token_spans"])
#sentences=review_sentences(list_tweet, remove_stopwords=True)
sentences=list_tweet
sentences

[['watching', 'for', 'bounce', 'tomorrow'],
 ['record', 'number', 'passengers', 'served'],
 ['nflx'],
 ['looking', 'for', 'strong', 'bounce', 'lunchtime', 'rally', 'coming'],
 ['intrigued', 'technology', 'growth', 'potential'],
 ['short', 'worked', 'puts', 'up'],
 ['overbought'],
 ['absolute',
  'garbage',
  'still',
  'up',
  'stores',
  'totally',
  'empty',
  'stock',
  'mispriced'],
 ['biggest', 'market', 'losers'],
 ['love', 'company', 'long', 'time'],
 ['goog', 'googl', 'would', 'suck'],
 ['wo', "n't", 'pay', 'anymore', 'real', 'risk'],
 ['edge', 'offered'],
 ['runs', 'sma', 'on', 'acquisition', 'news'],
 ['ca', "n't", 'go'],
 ['seems', 'like', 'helping', 'downtrend'],
 ['mastered', 'supply', 'chain'],
 ['bullish', 'stocks', 'on', 'twitter', 'dip'],
 ['big', 'dumping', 'would', 'not', 'touch', 'for'],
 ['trade', 'continuing', 'nicely', 'yesterday', 'looking', 'strong'],
 [],
 ['stochastic', 'overbought'],
 ['unusual', 'call', 'buying'],
 ['reserves', 'decline'],
 ['on', 'longerte

In [36]:
# Creating the model and setting values for the various parameters
num_features = 100  # Word vector dimensionality
min_word_count = 2 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "microblogs_word_vector"
model.save(model_name)
#model = gensim.models.Word2Vec.load(model_name)


2018-05-22 09:31:59,337 : INFO : collecting all words and their counts
2018-05-22 09:31:59,339 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-22 09:31:59,344 : INFO : collected 2218 word types from a corpus of 7608 raw words and 1700 sentences
2018-05-22 09:31:59,346 : INFO : Loading a fresh vocabulary
2018-05-22 09:31:59,351 : INFO : min_count=2 retains 1013 unique words (45% of original 2218, drops 1205)
2018-05-22 09:31:59,352 : INFO : min_count=2 leaves 6403 word corpus (84% of original 7608, drops 1205)
2018-05-22 09:31:59,357 : INFO : deleting the raw counts dictionary of 2218 items
2018-05-22 09:31:59,359 : INFO : sample=0.001 downsamples 66 most-common words
2018-05-22 09:31:59,360 : INFO : downsampling leaves estimated 5337 word corpus (83.4% of prior 6403)
2018-05-22 09:31:59,365 : INFO : estimated required memory for 1013 words and 100 dimensions: 1316900 bytes
2018-05-22 09:31:59,366 : INFO : resetting layer weights
2018-05-22 09:31:59,39

Training model....


2018-05-22 09:31:59,548 : INFO : saving Word2Vec object under microblogs_word_vector, separately None
2018-05-22 09:31:59,550 : INFO : not storing attribute vectors_norm
2018-05-22 09:31:59,551 : INFO : not storing attribute cum_table
2018-05-22 09:31:59,568 : INFO : saved microblogs_word_vector


In [37]:
# Few tests: This will print the odd word among them 
model.wv.doesnt_match("Thanks for the short upper".split())
len(model.wv.vocab)

2018-05-22 09:31:59,985 : WARNING : vectors for words {'the', 'Thanks'} are not present in the model, ignoring these words


1013

In [38]:
model.wv.most_similar("big")
#len(model.wv.index2word)

[('call', 0.4241793751716614),
 ('for', 0.3864457607269287),
 ('last', 0.36644792556762695),
 ('end', 0.35728055238723755),
 ('long', 0.35341405868530273),
 ('to', 0.35009369254112244),
 ('quarterly', 0.34859561920166016),
 ('keep', 0.3455084562301636),
 ('on', 0.33903056383132935),
 ('cuba', 0.33614590764045715)]

In [39]:
import numpy as np
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [40]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [41]:
#transformer les donner de train en moyenne de vecteurs word2vec 
trainDataVecs = getAvgFeatureVecs(list_tweet, model, num_features)

Review 0 of 1700
Review 1000 of 1700


C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [42]:
X=trainDataVecs


In [43]:
Y=data.processed_data["sentiment score"].values
Y

array([ 0.366,  0.638, -0.494, ...,  0.405,  0.296, -0.296])

In [44]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y,train_size=0.7)                                                  


In [45]:
#appliquer le mdele xgboost 
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
#Fitting XGB regressor 
model_xgb = xgb.XGBRegressor(max_depth=4, min_child_weight= 4,gamma=0.3,subsample=0.8,colsample_bytree=0.7)
model_xgb.fit(X,Y)
print (model_xgb)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=4, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)


In [46]:
data_test.do_process()
data_test.processed_data.head()

,cashtag,id,sentiment score,source,spans,token_spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short],"[putting, on, little, f, short]"
1,$AAPL,10752226,-0.464,stocktwits,[short some],[short]
2,$BAC,10920221,0.445,stocktwits,[buying opportunity],"[buying, opportunity]"
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position],"[scaling, up, on, long, position]"
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks],"[time, to, sell, banks]"


In [47]:
test_tweet=list(data_test.processed_data["token_spans"])

In [48]:
testDataVecs = getAvgFeatureVecs(test_tweet, model, num_features)

Review 0 of 16


C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [49]:
x_test=testDataVecs


In [50]:
#isoler les tweets du test et leur vrai valeur de sentiment 
labels=["real pred","spans"]
pred=list(data_test.processed_data["sentiment score"])
twt=list(data_test.processed_data["spans"])



In [51]:
#Predict 
output_2 = model_xgb.predict(x_test)
test_df = pd.DataFrame(pred,columns={"real prediction"})
#final_df["ID"] = id_vals
#final_df["cashtag"]=row2

test_df["spans"]=twt
test_df["Prediction"] = output_2
#final_df.to_csv("Output_1.csv",sep=",")
test_df.head(20)

,real prediction,spans,Prediction
0,-0.454,[Putting on a little $F short],0.016784
1,-0.464,[short some],-0.492857
2,0.445,[buying opportunity],0.087142
3,0.661,[Scaling Up on Long Position],0.275433
4,-0.763,[its time to sell banks],-0.437254
5,0.627,[Entering long],0.284053
6,0.653,[picked some up],0.190707
7,0.668,"[time to accumulate for a long position, far m...",0.287769
8,0.460,"[Looking for a strong bounce, Lunchtime rally ...",0.342317
9,0.403,[Very intrigued with the technology and growth...,0.233602


In [52]:
#tester la performance de prediction avec R2 score et MSE
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

print("mean squared error:" ,mean_squared_error(output_2, pred))
print("R2 score:" ,r2_score(output_2,pred))

mean squared error: 0.09462120484743503
R2 score: -0.16800682237999953
